## Problem Metadata 
DO NOT DELETE. This is a JSON Block. The metadata and course context MUST be in the first cell. 

```json
{
    "problem_title": "economizerHeatExchanger",
    "author": "tschutzius",
    "year": "Sp26",
    "difficulty_tags": ["easy"],
    "topic": "Exergy",
    "lecture_reference": {
        "lecture_id": "",
        "subtopics": [
            {
                "key": "",
                "topic": ""
            }
        ]
    },
    "book_reference": {
        "chapter_number": 8,
        "title": "Exergy",
        "sections": [
            "Exergy Balance: Control Volumes"
        ]
    },
    "notes": ""
}

## 1. Problem Statement
Chilled water is used to cool a computer-room air-handling (CRAH) unit. During favorable weather conditions or during night times, the data center cooling loads can be met entirely with the chilled water produced from a cooling tower alone. This eliminates the dependency on compressor-based cooling without affecting the temperature and humidity set points. During winter the economizer also serves as a redundant cooling system.

Air enters the CRAH with a flow rate of 10,000 CFM at 29$^\circ$C with 25% relative humidity and exits at 10$^\circ$C. The fan efficiency is 80% and the pressure drop across it is 250 Pa. Chilled water is pumped and enters and exits the CRAH at 7$^\circ$C and 15$^\circ$C, respectively. Cooling tower water is also pumped and enters and exits the economizer at 5$^\circ$C and 10$^\circ$C, respectively. The outside air temperature is 0$^\circ$C. The pressure head for the chilled water and cooling tower water are both 45 kPa and the pumps operate with 100% efficiency. 

**Goal:** Assuming that liquid water is incompressible, determine the following for the water-side economizer:
* (a) The rate of entropy generation.
* (b) The rate of exergy destruction.

## 2. Schematic 
```
     Water-side Economizer
     +------------------+
     |                  |
<--6-|                  |<-7-- (cooling tower water)
     |                  |
     +------------------+
     |                  |
+-5->|                  |-4->-+ (chilled water)
|    |                  |     | 
|    +------------------+     |
^                             v



^                             v
|           CRAH              |
|    +------------------+     |
|    |                  |     |
+-5--|                  |<-4--+ (chilled water)
     |                  |
     +------------------+
     |                  |
--1->|                  |--2-> (air)
     |                  |
     +------------------+
               |
               v
              (3) (water)
```
* State 1: air entering the CRAH
* State 2: air exiting the CRAH
* State 3: potentially liquid water exiting the CRAH
* State 4: chilled water entering the CRAH
* State 5: chilled water exiting the CRAH
* State 6: cooling tower entering the economizer.
* State 7: cooling tower exiting the economizer.

## 3. Assumptions and Approximations
* **Ideal gas mixture**: Water vapor and air behave as an ideal gas mixture

**Property Model and Analysis Method:**
* **Property tables: `pyCalor` (Accurate):**
        * **Purpose:** Must be used for determining water vapor and air properties. 
        * **Tools:** Properties ($h$, $s$, $T$, and $p$) are retrieved directly from a **thermodynamic property library** (e.g., **`pyCalor`**). Since it is a pure substance we only need to know two independent properties.

## 4. Physical Laws
**Mass balance:**
$$\dot{m}_\text{a1} = \dot{m}_\text{a2} = \dot{m}_\text{a} $$

$$\dot{m}_\text{a1}\omega_1 = \dot{m}_3 + \dot{m}_\text{a2}\omega_2$$

We can then write the amount of water leaving the CRAH as:

$$\dot{m}_3 = \dot{m}_\text{a} \left( \omega_1 - \omega_2 \right)$$

**Energy balance:**

$$\dot{m}_\text{a1}h_1 + \dot{W}_\text{fan} = \dot{m}_\text{a2}h_2 + \dot{m}_\text{3}h_3 + \dot{Q}_\text{out}$$

$$\dot{Q}_\text{out} = \dot{m}_\text{a1}h_1 + \dot{W}_\text{fan} - (\dot{m}_\text{a2}h_2 + \dot{m}_\text{3}h_3) $$

$$ \dot{W}_\text{fan} = \Delta p \dot{V} / \eta_\text{fan}$$

$$ \dot{W}_\text{pump} = \Delta p \dot{V} / \eta_\text{pump}$$



In [14]:
# 5. Properties (Code Cell)
try:
    from pyfluids import HumidAir, InputHumidAir
except ImportError:
    !pip install pyfluids
    from pyfluids import HumidAir, InputHumidAir

try:
    from CoolProp.CoolProp import PropsSI
except ImportError:
    !pip install CoolProp
    from CoolProp.CoolProp import PropsSI

try:
    from pyCalor import thermo as th
except ImportError:
    !pip install pyCalor
    from pyCalor import thermo as th

# dead state
T0_K = 273.15

# air flow rate
Vdot_CFM = 10000
Vdot_m3PerMin = Vdot_CFM/35.3147
Vdot_m3PerS = Vdot_m3PerMin/60

# fan efficiency, pressure drop
eta = 0.8
dp_Pa = 250

# pump efficiency, pressure drop
eta_p = 1
dp_Pa = 45e3

# state 1
T1_C = 29
RH1 = 25
p1_Pa = 101325

st1 = HumidAir().with_state(
InputHumidAir.pressure(p1_Pa),
InputHumidAir.temperature(T1_C),
InputHumidAir.relative_humidity(RH1),
#InputHumidAir.wet_bulb_temperature(Twb_C),
# InputHumidAir.dew_temperature(Tdp_C),
)

T1_dp_C = st1.dew_temperature; print("T1_dp_C=",T1_dp_C)
h1_JperKgHumidAir = st1.enthalpy; print("h1_JperKgHumidAir=",h1_JperKgHumidAir)
rho1_kgHumidAirPerM3 = st1.density; print("rho1_kgHumidAirPerM3=",rho1_kgHumidAirPerM3)
omega_1 = st1.humidity; print("omega_1=",omega_1)
h1_JperKgDryAir = h1_JperKgHumidAir*(1+omega_1); print("h1_JperKgDryAir=",h1_JperKgDryAir)
rho1_kgDryAirPerM3 = rho1_kgHumidAirPerM3/(1+omega_1); print("rho1_kgDryAirPerM3=",rho1_kgDryAirPerM3)

mdotAir_kgDryAirPerS = Vdot_m3PerS*rho1_kgDryAirPerM3; print("mdotAir_kgDryAirPerS=",mdotAir_kgDryAirPerS)

# state 2, T2 = 10 C
T2_C = 10
p2_Pa = p1_Pa
# since the dew point is at state 1 is less than T2, the absolute humidity hasn't changed
omega_2 = omega_1; print("omega_2 =",omega_2)

st2 = HumidAir().with_state(
InputHumidAir.pressure(p2_Pa),
InputHumidAir.temperature(T2_C),
InputHumidAir.humidity(omega_2)
#InputHumidAir.relative_humidity(RH1),
#InputHumidAir.wet_bulb_temperature(Twb_C),
# InputHumidAir.dew_temperature(Tdp_C),
)

h2_JperKgHumidAir = st2.enthalpy; print("h2_JperKgHumidAir=",h2_JperKgHumidAir)
h2_JperKgDryAir = h2_JperKgHumidAir*(1+omega_2); print("h2_JperKgDryAir=",h2_JperKgDryAir)

# state 3, for T2 = 10 c there is no water exiting

# state 4
substance = 'water'
T4_C = 7
p4_kPa = 101.325
st4 = th.state(substance,T=(T4_C,'C'),p=(p4_kPa,'kPa'),name='4')
print("h4_kJperKg =",st4.h )

# state 5
T5_C = 15
p5_kPa = p4_kPa
st5 = th.state(substance,T=(T5_C,'C'),p=(p5_kPa,'kPa'),name='5')
print("h5_kJperKg =",st5.h )

# state 6
T6_C = 5
p6_kPa = 101325
st6 = th.state(substance,T=(T6_C,'C'),p=(p6_kPa,'kPa'),name='6')
print("h6_kJperKg =",st6.h )

# state 7
T7_C = 10
p7_kPa = 101325
st7 = th.state(substance,T=(T7_C,'C'),p=(p7_kPa,'kPa'),name='7')
print("h7_kJperKg =",st7.h )

T1_dp_C= 7.00697358351033
h1_JperKgHumidAir= 44834.37451741706
rho1_kgHumidAirPerM3= 1.1642618189807894
omega_1= 0.006240871974970756
h1_JperKgDryAir= 45114.18010885815
rho1_kgDryAirPerM3= 1.1570408750100436
mdotAir_kgDryAirPerS= 5.460619680614793
omega_2 = 0.006240871974970756
h2_JperKgHumidAir= 25615.017587096074
h2_JperKgDryAir= 25774.877632493764
h4_kJperKg = 29.56752983566896
h5_kJperKg = 63.118715355870776
h6_kJperKg = 116.20754467824234
h7_kJperKg = 135.87111295316328


In [15]:
# 6 Calculations (Code Cell)
# WdotFan_kW = (dp_Pa * Vdot_m3PerS)/eta/1000
# print(f"(a) The power input to the fan is: {WdotFan_kW:.2f} kW")

QdotOut_kW = (mdotAir_kgDryAirPerS*(h1_JperKgDryAir - h2_JperKgDryAir) + WdotFan_kW)/1000
print(f"The heat transferred out of the CRAH for T2 = 10 C is: {QdotOut_kW:.2f} kW")

mdotCW_kgPerS = QdotOut_kW/(st5.h-st4.h)
print(f"The chilled water mass flow rate is: {mdotCW_kgPerS:.2f} kg/s")

mdotCTW_kgPerS = QdotOut_kW/(st7.h-st6.h)
print(f"The cooling tower water mass flow rate is: {mdotCTW_kgPerS:.2f} kg/s")

Sgen_Economizer_kWperK = mdotCTW_kgPerS*st7.s + mdotCW_kgPerS*st4.s - (mdotCTW_kgPerS*st6.s + mdotCW_kgPerS*st5.s)
print(f"The rate of entropy generation in the economizer is: {Sgen_Economizer_kWperK:.5f} kW/K")

XdotDestr_kW = Sgen_Economizer_kWperK*T0_K
print(f"The rate of exergy destruction is: {XdotDestr_kW:.5f} kW")

The heat transferred out of the CRAH for T2 = 10 C is: 105.87 kW
The chilled water mass flow rate is: 3.16 kg/s
The cooling tower water mass flow rate is: 5.38 kg/s
The rate of entropy generation in the economizer is: 0.00463 kW/K
The rate of exergy destruction is: 1.26377 kW


## 7. Summary and Reasoning

* **Key Insight:** 

THIS SHOULD BE THE FINAL CELL, DO NOT ADD OTHERS AFTER THIS, EVEN IF THEY'RE EMPTY
--- End of the Jupyter Notebook ---